In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv
import os
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [2]:
# 모델 초기화
model = ChatOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model="gpt-4o-mini")

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("resource/ai_trand.pdf")

# 페이지 별 문서 로드
docs = loader.load()
# CharacterTextSplitter 정의
text_splitter = CharacterTextSplitter(
    separator="\n\n",           # 각 청크를 구분하기 위한 기준 문자열
    chunk_size=100,             # 각 청크의 최대 길이
    chunk_overlap=10,           # 인접 청크 사이에 중복으로 포함될 문자수
    length_function=len,        # 청크의 길이를 계산하는 함수
    is_separator_regex=False,
)
splits = text_splitter.split_documents(docs)

# RecursiveCharacterTextSplitter 정의
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)
splits2 = recursive_text_splitter.split_documents(docs)

In [3]:
# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
# 효율적인 검색을 위해 retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])


In [4]:

# 사용자 질문이 그대로 전달되는지 확인하는 passthrough
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        # print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

while True: 
    print("========================")
    query = input("질문을 입력하세요: ")
    if query == "quit":
        break

    response = rag_chain_debug.invoke(query)
    print("Final Response:")
    print(response.content)

Final Response:
What would you like to know about the November 2024 issue?
Final Response:
'공공 부문의 AI를 위한 G7 툴킷' 보고서의 목적은 공공 부문에서 AI 도입을 위한 모범사례, 거버넌스 프레임워크, 정책 옵션을 제공하고, 안전하고 신뢰할 수 있는 AI의 원칙을 실행 가능한 정책으로 전환할 수 있도록 지원하는 것이다. 이를 통해 G7 회원국들이 공공 부문에서 AI를 효과적으로 활용할 수 있도록 단계별 접근방식을 권고하고 있다.


## 과제와 일반 GPT 답변의 차이를 확인하기 위한 질문
### 질문 : OECD에서 ai관련 보고서를 발간했다고 하는 '공공 부문의 AI를 위한 G7 툴킷' 보고서의 목적이 뭘까


## 과제의 질의 결과
##### '공공 부문의 AI를 위한 G7 툴킷' 보고서의 목적은 OECD가 G7 회원국이 작성한 설문 응답 및 연구를 기반으로 공공 부문에서 AI 활용에 대한 모범사례와 
##### 거버넌스 프레임워크, 정책 옵션을 제공하여, 안전하고 신뢰할 수 있는 AI의 원칙을 실행 가능한 정책으로 전환할 수 있도록 지원하는 것입니다.
##### 또한, 공공 부문 AI 도입을 단계별 접근 방식을 통해 안내하여 위험을 완화하고 효율성을 향상시키기 위한 것입니다.


## GPT 답변
##### The purpose of the OECD's "G7 Toolkit for Artificial Intelligence in the Public Sector" is to provide guidance and support to 
##### governments, particularly those in the G7, on how to effectively implement and govern AI technologies within the public sector. 
##### The toolkit emphasizes ensuring AI systems are trustworthy, secure, and operate transparently by promoting ethical standards, 
##### accountability, and data governance. It outlines strategies for managing AI's integration into public services,


## 결과
##### GPT의 응답결과도 영문이지만 해석해보면 LLM결과와 비슷한 답변을 가져온걸 알 수 있습니다.
##### 한국어로 이루어진 정보가 없는걸 보면 RAG를 활용해야 한글로된 정보를 얻기 편한것 같습니다.